In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.gmm import GMM
from datetime import datetime

In [2]:
data = pd.read_csv('bus1234.csv')
data = pd.read_csv('bus1234.csv')
data['day'] = 1
data['day'] = pd.to_datetime(data[['year','month','day']])

print(data.columns)

Index(['Unnamed: 0', 'id', 'group', 'year', 'month', 'replace', 'miles',
       'day'],
      dtype='object')


In [4]:
#from statsmodels.tsa.arima_model import ARMA

#create ID dummies
#different lines for each

#model = ARMA(data, order=(0, 1))
#model_fit = model.fit(disp=False)

In [27]:
class test(GMM):
    
    def __init__(self,*args, shift=50, gap=1, nmoms=10, **kwds, ):
        # set appropriate counts for moment conditions and parameters
        kwds.setdefault('k_moms', nmoms)
        kwds.setdefault('k_params',2)
        self.shift = shift
        super(test, self).__init__(*args, **kwds)
        
        
    def momcond(self, params):
        
        data = self.instrument.copy()
        data['miles'] = data['miles']/1e6
        data['util'] = params[0]*data['miles']*(1-data['replace']) + params[1]*data['replace']
        data['util_shock'] = data['shock'] + data['util']
        u_avg = data['util'].mean()
        
        moms = []
        for t in range(1,self.nmoms+1):
            #lag but loop around
            u_lag = data.groupby('id')['util'].shift(t*gap+self.shift)
            moms.append(data['util_shock']*u_lag - u_avg**2)
        moms = pd.DataFrame(np.array(moms).transpose())
        moms = moms.dropna()
        return moms
    
data['shock'] = sm.OLS(data['replace'], sm.add_constant( data['miles']/1e6 ) ).fit().resid
#data['shock'] = sm.Logit(data['replace'], sm.add_constant( data['miles']/1e6 ) ).fit().resid_response

data = data.sort_values(['id','day'])  
model = test(data['replace'],data['miles'],data[['id','miles','replace','day','shock']], shift=50)
result = model.fit(np.array([-.5,-10]),optim_method='nm')
result.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 54
         Function evaluations: 103
Optimization terminated successfully.
         Current function value: 0.000090
         Iterations: 20
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: 0.000093
         Iterations: 16
         Function evaluations: 33
Optimization terminated successfully.
         Current function value: 0.000093
         Iterations: 16
         Function evaluations: 33


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 test Results                                 
==============================================================================
Dep. Variable:                replace   Hansen J:                       0.2397
Model:                           test   Prob (Hansen J):                  1.00
Method:                           GMM                                         
Date:                Mon, 04 Nov 2019                                         
Time:                        19:36:36                                         
No. Observations:                8260                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0            0.0785      0.003     22.999      0.000       0.072       0.085
p 1           -0.9682      0.006   -167.266      0.000      -0.980      -0.957
==============================================================================
"""

In [41]:
N = 1000
ids = np.ones(N)
xs = np.zeros(N)
ys = np.zeros(N)
RC = -10
theta = -1



for i in range(1,N):

    ys[i] = (RC >= theta*xs[i-1] + np.random.uniform(-2,2) )
    
    if ys[i] == 0:
        xs[i] = xs[i-1] + np.random.uniform(0,1)
    if ys[i]==1:
        xs[i] = np.random.uniform(0,2)


        
data2 = pd.DataFrame( np.array([ids,xs,ys]).transpose(), columns=['id','miles','replace'])    
data2['shock'] = sm.OLS(data2['replace'], sm.add_constant( data2['miles']/1e6 ) ).fit().resid
#data['shock'] = sm.Logit(data['replace'], sm.add_constant( data['miles']/1e6 ) ).fit().resid_response
model = test(data2['replace'],data2['miles'],data2[['id','miles','replace','shock']] ,
             shift=100,gap=11,nmoms=4)
result = model.fit(np.array([-.5,-10]),optim_method='nm')
result.summary()

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 66
         Function evaluations: 126
Optimization terminated successfully.
         Current function value: 0.000749
         Iterations: 9
         Function evaluations: 18
Optimization terminated successfully.
         Current function value: 0.000009
         Iterations: 20
         Function evaluations: 39
Optimization terminated successfully.
         Current function value: 0.000023
         Iterations: 18
         Function evaluations: 34


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 test Results                                 
==============================================================================
Dep. Variable:                replace   Hansen J:                      0.02059
Model:                           test   Prob (Hansen J):                 0.990
Method:                           GMM                                         
Date:                Mon, 04 Nov 2019                                         
Time:                        19:47:54                                         
No. Observations:                1000                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0         7.731e-10   6.17e-09      0.125      0.900   -1.13e-08    1.29e-08
p 1         4.482e-13   1.53e-13      2.928      0.003    1.48e-13    7.48e-13
==============================================================================
"""